In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [8]:
subject = 'Distrito Federal - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [9]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_DF.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Produção de Cimento (t),Distrito Federal - Desemprego,Contratação Comercial (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao,Distrito Federal - Consumo de Cimento (t)
0,2008,1.257796e+08,3.618753e+06,44.111562,1.114321e+08,0.831284,0.850754,2685.923746,8.042711,5.483335e+08,...,26.827393,887.117978,2116.747379,21.422126,14.146668,66.792192,2.378486,0.847880,0.795527,909.713000
1,2009,1.266042e+08,3.629545e+06,44.141542,1.114727e+08,0.831790,0.851939,2935.357560,8.070885,5.302764e+08,...,26.876348,886.879495,1766.115242,21.441397,14.149368,66.806007,2.417640,0.846467,0.797514,991.263000
2,2010,1.296005e+08,3.625996e+06,44.116626,1.120321e+08,0.832181,0.852532,3227.460076,8.035310,5.046425e+08,...,26.495387,886.771930,1650.667839,21.298714,13.834925,66.431493,2.459251,0.845301,0.799450,1167.604000
3,2011,1.323025e+08,3.587906e+06,44.022754,1.124103e+08,0.832459,0.852542,3532.866309,8.024653,4.854622e+08,...,26.497614,886.883230,1564.062453,21.274581,13.773402,66.359941,2.445634,0.844378,0.801325,1277.890000
4,2012,1.348108e+08,3.550906e+06,43.926397,1.127652e+08,0.832380,0.851897,3799.093892,10.763245,4.867661e+08,...,27.337584,886.989319,1529.804524,21.318450,13.817656,66.310747,2.408201,0.843623,0.802661,1261.667000
5,2013,1.372002e+08,3.512366e+06,43.880001,1.131945e+08,0.832995,0.850805,3818.436361,11.341796,5.225848e+08,...,27.062026,887.169899,1507.055019,21.353309,13.761380,66.172806,2.347696,0.843425,0.805550,1152.069000
6,2014,1.392692e+08,3.476698e+06,43.883670,1.135587e+08,0.832661,0.849205,3551.813908,11.799788,5.676413e+08,...,26.037094,887.401648,1507.000310,21.640954,13.621200,64.854756,2.309168,0.843044,0.806512,979.998857
7,2015,1.407164e+08,3.438025e+06,43.849398,1.137031e+08,0.832383,0.847056,3186.773594,12.093705,5.179392e+08,...,25.992461,887.544811,1513.172948,21.797552,13.512417,63.666484,2.318908,0.842881,0.807805,811.194686
8,2016,1.416229e+08,3.431106e+06,43.828558,1.137585e+08,0.834584,0.854633,2900.209377,12.238790,4.524144e+08,...,27.482499,887.589041,1508.330179,21.640241,13.367470,63.456555,2.339247,0.836089,0.813684,638.818457
9,2017,1.417351e+08,3.426736e+06,43.802546,1.136558e+08,0.833455,0.851963,2730.619877,12.085632,4.213689e+08,...,27.991365,887.652320,1501.483221,21.558960,13.478044,64.168678,2.297251,0.836097,0.812928,584.646000


In [10]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Produção de Cimento (t),Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
0,1.257796e+08,3.618753e+06,44.111562,1.114321e+08,0.831284,0.850754,2685.923746,8.042711,5.483335e+08,2.546644e+09,...,11.760983,26.827393,887.117978,2116.747379,21.422126,14.146668,66.792192,2.378486,0.847880,0.795527
1,1.266042e+08,3.629545e+06,44.141542,1.114727e+08,0.831790,0.851939,2935.357560,8.070885,5.302764e+08,2.679945e+09,...,9.472585,26.876348,886.879495,1766.115242,21.441397,14.149368,66.806007,2.417640,0.846467,0.797514
2,1.296005e+08,3.625996e+06,44.116626,1.120321e+08,0.832181,0.852532,3227.460076,8.035310,5.046425e+08,2.891930e+09,...,9.332485,26.495387,886.771930,1650.667839,21.298714,13.834925,66.431493,2.459251,0.845301,0.799450
3,1.323025e+08,3.587906e+06,44.022754,1.124103e+08,0.832459,0.852542,3532.866309,8.024653,4.854622e+08,3.091988e+09,...,10.985689,26.497614,886.883230,1564.062453,21.274581,13.773402,66.359941,2.445634,0.844378,0.801325
4,1.348108e+08,3.550906e+06,43.926397,1.127652e+08,0.832380,0.851897,3799.093892,10.763245,4.867661e+08,3.189506e+09,...,8.159014,27.337584,886.989319,1529.804524,21.318450,13.817656,66.310747,2.408201,0.843623,0.802661
5,1.372002e+08,3.512366e+06,43.880001,1.131945e+08,0.832995,0.850805,3818.436361,11.341796,5.225848e+08,3.093872e+09,...,7.913561,27.062026,887.169899,1507.055019,21.353309,13.761380,66.172806,2.347696,0.843425,0.805550
6,1.392692e+08,3.476698e+06,43.883670,1.135587e+08,0.832661,0.849205,3551.813908,11.799788,5.676413e+08,2.779340e+09,...,10.350442,26.037094,887.401648,1507.000310,21.640954,13.621200,64.854756,2.309168,0.843044,0.806512
7,1.407164e+08,3.438025e+06,43.849398,1.137031e+08,0.832383,0.847056,3186.773594,12.093705,5.179392e+08,2.561731e+09,...,12.454398,25.992461,887.544811,1513.172948,21.797552,13.512417,63.666484,2.318908,0.842881,0.807805
8,1.416229e+08,3.431106e+06,43.828558,1.137585e+08,0.834584,0.854633,2900.209377,12.238790,4.524144e+08,2.516859e+09,...,13.111908,27.482499,887.589041,1508.330179,21.640241,13.367470,63.456555,2.339247,0.836089,0.813684
9,1.417351e+08,3.426736e+06,43.802546,1.136558e+08,0.833455,0.851963,2730.619877,12.085632,4.213689e+08,2.497710e+09,...,9.479812,27.991365,887.652320,1501.483221,21.558960,13.478044,64.168678,2.297251,0.836097,0.812928


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      991.263000
1     1167.604000
2     1277.890000
3     1261.667000
4     1152.069000
5      979.998857
6      811.194686
7      638.818457
8      584.646000
9      571.130000
10     637.028000
11     662.625000
12     789.947000
13     786.865000
14            NaN
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Produção de Cimento (t),Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
0,-1.772344,1.305688,1.394520,-1.771485,-1.481198,0.073029,-1.093654,-1.382842,1.204752,-0.990234,...,0.984445,-0.389874,-0.480815,2.969931,-0.431194,1.774968,1.163601,0.406482,1.415122,-1.596560
1,-1.628397,1.445977,1.617168,-1.721807,-0.881748,0.527711,-0.499779,-1.367272,0.820657,-0.546688,...,-0.051710,-0.353331,-1.201863,0.970958,-0.308465,1.786791,1.175566,0.949406,1.092023,-1.294324
2,-1.105388,1.399843,1.432130,-1.038286,-0.418247,0.754975,0.195686,-1.386932,0.275392,0.158672,...,-0.115145,-0.637710,-1.527088,0.312786,-1.217153,0.409447,0.851180,1.526394,0.825602,-0.999913
3,-0.633732,0.904674,0.734968,-0.576169,-0.088519,0.758903,0.922826,-1.392822,-0.132596,0.824346,...,0.633403,-0.636048,-1.190571,-0.180956,-1.370845,0.139961,0.789204,1.337570,0.614654,-0.714710
4,-0.195890,0.423679,0.019359,-0.142567,-0.182504,0.511487,1.556686,0.120653,-0.104860,1.148829,...,-0.646476,-0.009027,-0.869813,-0.376263,-1.091464,0.333806,0.746595,0.818513,0.442192,-0.511457
5,0.221192,-0.077344,-0.325210,0.381968,0.546412,0.092602,1.602738,0.440388,0.657046,0.830615,...,-0.757614,-0.214725,-0.323831,-0.505959,-0.869457,0.087303,0.627117,-0.020457,0.396927,-0.072147
6,0.582347,-0.541036,-0.297957,0.827000,0.150865,-0.521032,0.967938,0.693496,1.615453,-0.215960,...,0.345772,-0.979817,0.376858,-0.506271,0.962428,-0.526723,-0.514518,-0.554697,0.309796,0.074238
7,0.834959,-1.043781,-0.552484,1.003356,-0.178876,-1.345643,0.098816,0.855928,0.558230,-0.940033,...,1.298413,-1.013134,0.809710,-0.471080,1.959734,-1.003221,-1.543744,-0.419639,0.272544,0.270845
8,0.993196,-1.133735,-0.707261,1.071087,2.429074,1.561122,-0.583463,0.936109,-0.835563,-1.089341,...,1.596124,0.099149,0.943440,-0.498689,0.957885,-1.638128,-1.725576,-0.137615,-1.279978,1.165081
9,1.012775,-1.190536,-0.900442,0.945623,1.091262,0.536962,-0.987238,0.851467,-1.495939,-1.153056,...,-0.048438,0.479007,1.134763,-0.537724,0.440241,-1.153784,-1.108767,-0.719942,-1.278138,1.050104


In [13]:
reshaped_train_input = dfToInputRNN(train_input)

In [14]:
# validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
# validation_input = dfToInputRNN(validation_input)
# validation_input

In [15]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      991.263000
1     1167.604000
2     1277.890000
3     1261.667000
4     1152.069000
5      979.998857
6      811.194686
7      638.818457
8      584.646000
9      571.130000
10     637.028000
11     662.625000
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [20]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    662.625
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [21]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Produção de Cimento (t),Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
12,0.37302,-0.228675,-1.817412,-0.320956,-2.605142,-2.578139,-0.539067,0.722486,1.108405,2.635823,...,-2.260447,2.753929,0.526707,-0.506307,-0.465998,-0.789616,-0.276498,-1.706569,-1.348539,0.91254


In [22]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.3730197 , -0.2286745 , -1.81741216, -0.32095639,
         -2.60514151, -2.5781393 , -0.53906659,  0.72248569,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738,  2

In [23]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    789.947
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [24]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [25]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [26]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[859003893, 1009787905, 2870178212, 4191095213, 2048085337, 1432464263, 616372699, 4162282062, 4073028276, 2179442668, 2695462413, 769272132, 388406210, 2062079210, 2658340363, 4081665592, 1908811431, 3351392920, 1776824128, 495637605, 2631955045, 1990677391, 332914571, 1720595267, 2030148817, 1378059951, 2849527233, 1445221376, 2259642618, 1965497736, 1751002221, 2820345207, 1800727523, 3777704175, 4174266095, 3141672479, 846576309, 2735723428, 3806143345, 3930624473, 3908746765, 3161743418, 3850517858, 2864597001, 88975906, 2356227146, 4184023399, 1004976708, 1803038314, 910246175, 2320557203, 574659834, 1942518236, 1877766038, 3677500355, 2999500770, 2712294407, 2646603202, 415898919, 3252175836, 13909821, 1622161308, 562159953, 388342635, 1091494279, 158659135, 2834960897, 2856634178, 1498205137, 3790380886, 3421009232, 3053329093, 3151918660, 4248843889, 1496283963, 281442470, 4035785503, 1378552175, 1602430766, 461112586, 505367180, 1842186065, 2895644236, 98129936, 502413495, 32

2023-09-08 16:13:35.666837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 16:13:35.686287: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 16:13:35.686445: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 1295.7650146484375
winner_seed: 859003893


Step: 1 ___________________________________________
loss: 1601.85693359375


Step: 2 ___________________________________________
loss: 1549.1859130859375


Step: 3 ___________________________________________
loss: 1307.1448974609375


Step: 4 ___________________________________________
loss: 1703.3414306640625


Step: 5 ___________________________________________
loss: 1681.4862060546875


Step: 6 ___________________________________________
loss: 1476.2718505859375


Step: 7 ___________________________________________
loss: 1185.2781982421875
winner_seed: 4162282062


Step: 8 ___________________________________________
loss: 1299.3306884765625


Step: 9 ___________________________________________
loss: 1474.8367919921875


Step: 10 ___________________________________________
loss: 1473.5494384765625


Step: 11 ___________________________________________
loss: 1251.201904296875


Step: 12 ___________________________________________
loss:

In [27]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 869773.0000
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 831849.6875
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 474565.6562
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 255095.8906
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 677770.3125
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 178685.9219
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 112584.0391
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 12890.8057
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 10756.5000
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 8471.4434
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 11174.7695
Epoch 12/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 9020.7705
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 11976.4346
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 8351.0000
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 8744.3721
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 4677.9780
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 10496.2588
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 9197.0400
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 13726.5205
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 3228.8865
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 9482.4463
Epoch 108/10000
1/1 [==============================] - 0s 5ms/step - loss: 12973.1475
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 459

1/1 [==============================] - 0s 4ms/step - loss: 5309.5493
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 7080.8359
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 6675.3491
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 9107.0088
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 7679.6919
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 8590.7764
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 7397.6646
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 9050.5146
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 8642.1377
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 5284.6694
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 5928.9185
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 10017.

1/1 [==============================] - 0s 4ms/step - loss: 8691.1523
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 6534.9204
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 10476.3486
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 11801.4756
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 6343.2466
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 5978.4663
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 5884.9692
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 7672.7715
Epoch 298/10000
1/1 [==============================] - 0s 5ms/step - loss: 6680.1172
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 10745.0117
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 12685.8174
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 13

1/1 [==============================] - 0s 4ms/step - loss: 3513.5012
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 6985.2065
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 8277.3135
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 6775.9883
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 8727.7744
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 7117.5972
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 7616.0371
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - loss: 6773.5391
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 11265.0430
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 6732.1313
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 8513.9189
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 10306

...

1/1 [==============================] - 0s 4ms/step - loss: 12732.8477
Epoch 675/10000
1/1 [==============================] - 0s 4ms/step - loss: 7256.5093
Epoch 676/10000
1/1 [==============================] - 0s 4ms/step - loss: 8572.1084
Epoch 677/10000
1/1 [==============================] - 0s 4ms/step - loss: 4824.9673
Epoch 678/10000
1/1 [==============================] - 0s 4ms/step - loss: 7611.9009
Epoch 679/10000
1/1 [==============================] - 0s 4ms/step - loss: 5639.1733
Epoch 680/10000
1/1 [==============================] - 0s 4ms/step - loss: 4885.6523
Epoch 681/10000
1/1 [==============================] - 0s 4ms/step - loss: 3586.1511
Epoch 682/10000
1/1 [==============================] - 0s 4ms/step - loss: 7857.8906
Epoch 683/10000
1/1 [==============================] - 0s 4ms/step - loss: 7817.9395
Epoch 684/10000
1/1 [==============================] - 0s 4ms/step - loss: 9968.3994
Epoch 685/10000
1/1 [==============================] - 0s 4ms/step - loss: 9505.

1/1 [==============================] - 0s 4ms/step - loss: 5545.5757
Epoch 771/10000
1/1 [==============================] - 0s 4ms/step - loss: 4433.6206
Epoch 772/10000
1/1 [==============================] - 0s 4ms/step - loss: 4983.5151
Epoch 773/10000
1/1 [==============================] - 0s 3ms/step - loss: 4369.3379
Epoch 774/10000
1/1 [==============================] - 0s 4ms/step - loss: 5957.7593
Epoch 775/10000
1/1 [==============================] - 0s 4ms/step - loss: 4670.1870
Epoch 776/10000
1/1 [==============================] - 0s 4ms/step - loss: 8497.1143
Epoch 777/10000
1/1 [==============================] - 0s 4ms/step - loss: 12923.4873
Epoch 778/10000
1/1 [==============================] - 0s 4ms/step - loss: 4827.1284
Epoch 779/10000
1/1 [==============================] - 0s 4ms/step - loss: 5924.6899
Epoch 780/10000
1/1 [==============================] - 0s 4ms/step - loss: 6527.6895
Epoch 781/10000
1/1 [==============================] - 0s 4ms/step - loss: 4661.

1/1 [==============================] - 0s 4ms/step - loss: 7082.6089
Epoch 867/10000
1/1 [==============================] - 0s 4ms/step - loss: 6252.9238
Epoch 868/10000
1/1 [==============================] - 0s 4ms/step - loss: 6653.3940
Epoch 869/10000
1/1 [==============================] - 0s 4ms/step - loss: 12224.0693
Epoch 870/10000
1/1 [==============================] - 0s 4ms/step - loss: 8426.5576
Epoch 871/10000
1/1 [==============================] - 0s 4ms/step - loss: 3061.1797
Epoch 872/10000
1/1 [==============================] - 0s 4ms/step - loss: 5539.6313
Epoch 873/10000
1/1 [==============================] - 0s 4ms/step - loss: 6143.3013
Epoch 874/10000
1/1 [==============================] - 0s 4ms/step - loss: 4822.1782
Epoch 875/10000
1/1 [==============================] - 0s 4ms/step - loss: 15449.5947
Epoch 876/10000
1/1 [==============================] - 0s 4ms/step - loss: 7017.5962
Epoch 877/10000
1/1 [==============================] - 0s 4ms/step - loss: 9726

1/1 [==============================] - 0s 4ms/step - loss: 6311.2500
Epoch 963/10000
1/1 [==============================] - 0s 4ms/step - loss: 5097.6675
Epoch 964/10000
1/1 [==============================] - 0s 4ms/step - loss: 6337.7852
Epoch 965/10000
1/1 [==============================] - 0s 4ms/step - loss: 6193.6406
Epoch 966/10000
1/1 [==============================] - 0s 4ms/step - loss: 5211.1533
Epoch 967/10000
1/1 [==============================] - 0s 4ms/step - loss: 7961.5820
Epoch 968/10000
1/1 [==============================] - 0s 4ms/step - loss: 4018.5510
Epoch 969/10000
1/1 [==============================] - 0s 4ms/step - loss: 7643.0640
Epoch 970/10000
1/1 [==============================] - 0s 4ms/step - loss: 4928.4453
Epoch 971/10000
1/1 [==============================] - 0s 4ms/step - loss: 9135.8271
Epoch 972/10000
1/1 [==============================] - 0s 4ms/step - loss: 5469.2017
Epoch 973/10000
1/1 [==============================] - 0s 4ms/step - loss: 9001.1

1/1 [==============================] - 0s 4ms/step - loss: 4572.9502
Epoch 1058/10000
1/1 [==============================] - 0s 4ms/step - loss: 4451.8403
Epoch 1059/10000
1/1 [==============================] - 0s 4ms/step - loss: 6340.4858
Epoch 1060/10000
1/1 [==============================] - 0s 4ms/step - loss: 6039.8530
Epoch 1061/10000
1/1 [==============================] - 0s 4ms/step - loss: 4080.4329
Epoch 1062/10000
1/1 [==============================] - 0s 4ms/step - loss: 5824.3516
Epoch 1063/10000
1/1 [==============================] - 0s 4ms/step - loss: 8594.5117
Epoch 1064/10000
1/1 [==============================] - 0s 4ms/step - loss: 6467.2578
Epoch 1065/10000
1/1 [==============================] - 0s 4ms/step - loss: 5168.2339
Epoch 1066/10000
1/1 [==============================] - 0s 4ms/step - loss: 8903.2939
Epoch 1067/10000
1/1 [==============================] - 0s 4ms/step - loss: 7673.1509
Epoch 1068/10000
1/1 [==============================] - 0s 4ms/step - l

In [28]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 593ms/step
[789.947](test_target) - [631.92737](prediction) = 158.0196318359375


In [29]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.20003827071428526

In [30]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [31]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Produção de Cimento (t),Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
0,-0.943236,-1.338597,-0.891494,-0.780055,-1.273491,-1.336345,-1.191346,-0.451783,1.148205,-1.122043,...,1.412346,0.557520,1.347425,1.373641,0.549175,0.697943,0.666894,-1.212140,1.262053,-1.230064
1,-0.440922,1.064416,1.396499,-0.631558,0.104144,0.267379,-0.064267,1.386460,0.140893,-0.184474,...,-0.643250,0.846797,-0.301781,-0.395561,0.854043,0.716231,0.746571,-0.024832,-0.078381,0.010708
2,1.384158,0.274181,-0.505004,1.411612,1.169347,1.068967,1.255613,-0.934677,-1.289098,1.306518,...,-0.769096,-1.404317,-1.045645,-0.978080,-1.403217,-1.414174,-1.413465,1.236972,-1.183672,1.219356


train_target:


,Distrito Federal - Consumo de Cimento (t)
0,991.263
1,1167.604
2,1277.890


test_input:


...

,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - IDH,Distrito Federal - IDH Longevidade,Distrito Federal - Produção de Cimento (t),Distrito Federal - Desemprego,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Distrito Federal - IDH Renda,Distrito Federal - IDH Educacao
12,0.37302,-0.228675,-1.817412,-0.320956,-2.605142,-2.578139,-0.539067,0.722486,1.108405,2.635823,...,-2.260447,2.753929,0.526707,-0.506307,-0.465998,-0.789616,-0.276498,-1.706569,-1.348539,0.91254


test_target:


,Distrito Federal - Consumo de Cimento (t)
12,789.947


1/1 [==============================] - 1s 1s/step
Error: 158.0196318359375




[1235.3275146484375,
 1268.9888916015625,
 1131.236328125,
 844.9896240234375,
 832.8386840820312,
 656.3193359375,
 621.2505493164062,
 583.2406616210938,
 608.4534301757812,
 631.9273681640625]

In [32]:
display(targets)
display(predictions)

[1261.667,
 1152.069,
 979.9988571428572,
 811.1946857142857,
 638.8184571428571,
 584.646,
 571.13,
 637.028,
 662.625,
 789.947]

[1235.3275146484375,
 1268.9888916015625,
 1131.236328125,
 844.9896240234375,
 832.8386840820312,
 656.3193359375,
 621.2505493164062,
 583.2406616210938,
 608.4534301757812,
 631.9273681640625]

In [33]:
mae = mean_absolute_error(predictions, targets)
mae

91.00844384765625

In [34]:
porcentage = mae/np.mean(targets)
porcentage

0.11250716869670468